In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Conduct word2vec and symbolic feature experiments in combination with Linear SVC
* Linear SVC experiments are conducted in this notebook with Doc2Vec features and BOW TF TFIDF and balanced versions of those.
* Hyperparameter grid search tuning in combination with a k-fold cross validation. 
* Note: You need to get in reach of a feature generated set to run this notebook.




In [0]:
a = [] #RAM trick google Colab
while(1):
  a.append('1')

In [0]:
import pandas as pd #import 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm, metrics, model_selection
from ast import literal_eval
from sklearn.metrics import classification_report
import time
from sklearn.multiclass import OneVsRestClassifier
import statistics
from sklearn.svm import LinearSVC

In [0]:
def train_test_valid_split(df,upsampling, print_distribution = False): #split train, valid and test reasonably, balance the set if declared so
    train,test = train_test_split(df,test_size=0.3,stratify=df["Reviewer_Score"], random_state=42)
    test,valid = train_test_split(test,test_size=0.5,stratify=test["Reviewer_Score"], random_state=42)
    #Zusammengefasst, folgende Aufteilung:
    #70% Training, 15% Validation, 15% Test


    unique, counts_train = np.unique(train["Reviewer_Score"], return_counts=True)
    if(upsampling): #Idee: reduce class 0 to the size of class 1, dupliate samples from class 2 to the size of class 1
        train_0 = train[train["Reviewer_Score"]==0].sample(frac=(counts_train[1]/counts_train[0]), random_state=42)
        train_1 = train[train["Reviewer_Score"]==1]
        train_2 = train[train["Reviewer_Score"]==2].sample(frac=(counts_train[1]/counts_train[2]),replace=True, random_state=42)
        train = train_0.append(train_1).append(train_2)
        train = train.sample(frac=1, random_state=42)
    
    if (print_distribution):
      unique, counts_train = np.unique(train["Reviewer_Score"], return_counts=True)
      plt.bar(unique, counts_train)
      unique, counts = np.unique(test["Reviewer_Score"], return_counts=True)
      plt.bar(unique, counts)
      unique, counts = np.unique(valid["Reviewer_Score"], return_counts=True)
      plt.bar(unique, counts)
      plt.title('Class Frequency')
      plt.xlabel('Class')
      plt.ylabel('Frequency')
      plt.show()
    
    return train,valid,test

def load_from_source(): #load data
  # list that holds a list for each category of features with the file paths for the feature data
  # "/BOW/tf-idf-set-0-005-tokenization-stpwrds-stemming.csv", "/BOW/tf-idf-set-0-005-tokenization-stpwrds-stemming-ngram.csv"
  feature_filepaths=[["/BOW/tf_561-woerter.pkl", "/BOW/tfidf_561-woerter.pkl"],["/doc2vec/Pretrained_withScore.csv", "/doc2vec/Owntrained_withScore.csv"]]
  # list that holds a list for each category of features with the labels for the feature data, fill this in the same way the filepath array is filled
  feature_labels = [["fast text without stop-word removal", "fast text with stop-word removal"],["Doc2Vec Pretrained","Doc2Vec Owntrained"]]
  # Load the dataframes and safe them in the same structure like the filepath and labels
  dataframes = []
  for feature_type_filepaths in feature_filepaths:
    feature_type_dataframes = []
    for feature_filepath in feature_type_filepaths:
      if feature_filepath[-3:] == "csv":
        df =  pd.read_csv("/content/drive/My Drive/Feature_generated_sets" + feature_filepath)
        if 'Unnamed: 0' in df.columns:
          df = df.drop('Unnamed: 0', 1)
      if feature_filepath[-3:] == "pkl":
        df =  pd.read_pickle("/content/drive/My Drive/Feature_generated_sets" + feature_filepath)
        if 'Unnamed: 0' in df.columns:
          df = df.drop('Unnamed: 0', 1)
      feature_type_dataframes.append(df)
    dataframes.append(feature_type_dataframes)
  return dataframes


def split_dataframes(dataframes,upsampling,test_boolean=False,test_size=100000000): #organizing the whole structure in arrays and dictionaries
  # split the dataframes with the upper method and save them in a dictonary in arrays like the filepath
  test_samples = lambda df: df[0:test_size] if test_boolean else df
  split_dataframes = [] 
  for feature_type_dataframes in dataframes:
    feature_type_split_dataframes = []
    for feature_data in feature_type_dataframes:
      train, valid, test = train_test_valid_split(feature_data, upsampling)
      train, valid, test = test_samples(train),test_samples(valid), test_samples(test)
      feature_type_split_dataframes.append({"train": train, "valid": valid, "test":test}) 

    split_dataframes.append(feature_type_split_dataframes)

  return split_dataframes

def trans_to_numpy_split(split_dataframes): #correcting data formats if necesarry for sklearn
  # transform data to numpy arrays
  split_types = ["train", "valid", "test"]
  for feature_type_dataframes in split_dataframes:
    for feature_data in feature_type_dataframes:
      if len(feature_data["train"].columns) == 2:
        for st in split_types:
          features = np.array(feature_data[st]["Review"].tolist())
          label = np.array(feature_data[st]["Reviewer_Score"].tolist())
          feature_data[st] = {"features": features, "label": label}
      elif len(feature_data["train"].columns) == 301:
        for st in split_types:
          features = np.array(feature_data[st].loc[:, :'299'].values)
          label = np.array(feature_data[st]["Reviewer_Score"].values)
          feature_data[st] = {"features": features, "label": label}
      elif len(feature_data["train"].columns) == 562:
        for st in split_types:
          features = np.array(feature_data[st].loc[:, :'yet'].values)
          label = np.array(feature_data[st]["Reviewer_Score"].values)
          feature_data[st] = {"features": features, "label": label}
  return split_dataframes

def trans_to_numpy_unsplit(dataframes, test, test_size): #assisting method to reorganize data formats
  # transform data to numpy arrays
  df_result = []
  for feature_type_dataframes in dataframes:
    df_types = []
    for feature_data in feature_type_dataframes:
      if test:
        feature_data = feature_data[:test_size]
      if len(feature_data.columns) == 2:
        features = np.array(feature_data["Review"].tolist())
        label = np.array(feature_data["Reviewer_Score"].tolist())
        feature_data = {"features": features, "label": label}
      elif len(feature_data.columns) == 301:
        features = np.array(feature_data.loc[:, :'299'].values)
        label = np.array(feature_data["Reviewer_Score"].values)
        feature_data = {"features": features, "label": label}
      elif len(feature_data.columns) == 562:
        features = np.array(feature_data.loc[:, :'yet'].values)
        label = np.array(feature_data["Reviewer_Score"].values)
        feature_data = {"features": features, "label": label}
      df_types.append(feature_data)
    df_result.append(df_types)
  return df_result

In [0]:
dataframes = load_from_source()

In [0]:
def cross_validation_hyperparameter_grid_search_for_SVM(dataframes, upsampling, i, j, description, param_grid={ 'dual' : [False] ,"tol": [0.00001,0.0001,0.001,0.01], 'C': [0.1,5,10], "multi_class": ["ovr","crammer_singer"]}): #automatic grid search implementation
  
  dicts = trans_to_numpy_split(split_dataframes(dataframes, upsampling, test_boolean=False, test_size=20000))
  dicts = dicts[i][j]
  train_x, train_y, valid_x, valid_y, test_x, test_y = dicts["train"]["features"],dicts["train"]["label"], dicts["valid"]["features"],dicts["valid"]["label"], dicts["test"]["features"],dicts["test"]["label"]



  svc = svm.LinearSVC()
  param_grid = { 'dual' : [False] ,"tol": [0.00001,0.0001,0.001,0.01], 'C': [0.1,5,10], "multi_class": ["ovr","crammer_singer"]}
  grid_svm = model_selection.GridSearchCV(svc,
                      param_grid= param_grid, 
                      scoring="f1_macro",
                      cv=4,   
                      n_jobs=-1) 
  print("Training now: ",description)
  print("Balanced: ",upsampling)
  time1 = time.time()
  grid_svm.fit(train_x, train_y)
  print(time.time()-time1)
  # from https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html
  print("Best parameters set found on development set:")
  print(grid_svm.best_params_,"\n")
  
  print("The model is trained on the full development set.")
  print("The scores are computed on the full evaluation set.\n")
  print("Detailed classification report:")
  y_true, y_pred = test_y, grid_svm.predict(test_x)
  print(metrics.classification_report(y_true, y_pred),"\n")
  print("-----------------------------------------------------")



In [0]:
# Results

Training now:  /BOW/tf_561-woerter.pkl
Balanced:  True
972.0990505218506
Best parameters set found on development set:
{'C': 5, 'dual': False, 'multi_class': 'ovr', 'tol': 0.01} 

The model is trained on the full development set.
The scores are computed on the full evaluation set.

Detailed classification report:
              precision    recall  f1-score   support

           0       0.81      0.70      0.75     44096
           1       0.41      0.38      0.39     20237
           2       0.47      0.74      0.58     13028

    accuracy                           0.62     77361
   macro avg       0.56      0.61      0.58     77361
weighted avg       0.65      0.62      0.63     77361
 

-----------------------------------------------------
Training now:  /BOW/tf_561-woerter.pkl
Balanced:  False
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
1156.450232744217
Best parameters set found on development set:
{'C': 10, 'dual': False, 'multi_class': 'ovr', 'tol': 0.001} 

The model is trained on the full development set.
The scores are computed on the full evaluation set.

Detailed classification report:
              precision    recall  f1-score   support

           0       0.71      0.91      0.80     44096
           1       0.46      0.19      0.27     20237
           2       0.60      0.56      0.58     13028

    accuracy                           0.66     77361
   macro avg       0.59      0.55      0.55     77361
weighted avg       0.62      0.66      0.62     77361
 

-----------------------------------------------------
Training now:  /BOW/tfidf_561-woerter.pkl
Balanced:  True
956.3361351490021
Best parameters set found on development set:
{'C': 0.1, 'dual': False, 'multi_class': 'ovr', 'tol': 0.001} 

The model is trained on the full development set.
The scores are computed on the full evaluation set.

Detailed classification report:
              precision    recall  f1-score   support

           0       0.82      0.70      0.75     44096
           1       0.41      0.38      0.39     20237
           2       0.47      0.74      0.57     13028

    accuracy                           0.62     77361
   macro avg       0.56      0.61      0.57     77361
weighted avg       0.65      0.62      0.63     77361
 

-----------------------------------------------------
Training now:  /BOW/tfidf_561-woerter.pkl
Balanced:  False
1132.4396421909332
Best parameters set found on development set:
{'C': 5, 'dual': False, 'multi_class': 'ovr', 'tol': 0.001} 

The model is trained on the full development set.
The scores are computed on the full evaluation set.

Detailed classification report:
              precision    recall  f1-score   support

           0       0.70      0.91      0.80     44096
           1       0.46      0.18      0.26     20237
           2       0.59      0.56      0.57     13028

    accuracy                           0.66     77361
   macro avg       0.59      0.55      0.54     77361
weighted avg       0.62      0.66      0.62     77361
 

-----------------------------------------------------
Training now:  /doc2vec/Pretrained_withScore.csv
Balanced:  True
25041.50695824623
Best parameters set found on development set:
{'C': 5, 'dual': False, 'multi_class': 'ovr', 'tol': 0.0001} 

The model is trained on the full development set.
The scores are computed on the full evaluation set.

Detailed classification report:
              precision    recall  f1-score   support

           0       0.79      0.73      0.76     44096
           1       0.41      0.34      0.37     20237
           2       0.46      0.72      0.56     13028

    accuracy                           0.62     77361
   macro avg       0.56      0.59      0.57     77361
weighted avg       0.64      0.62      0.62     77361
 

-----------------------------------------------------
Training now:  /doc2vec/Owntrained_withScore.csv
Balanced:  True
1420.5369265079498
Best parameters set found on development set:
{'C': 10, 'dual': False, 'multi_class': 'ovr', 'tol': 1e-05} 

The model is trained on the full development set.
The scores are computed on the full evaluation set.

Detailed classification report:
              precision    recall  f1-score   support

           0       0.57      0.36      0.44     44096
           1       0.26      0.30      0.28     20237
           2       0.17      0.35      0.23     13028

    accuracy                           0.34     77361
   macro avg       0.34      0.34      0.32     77361
weighted avg       0.42      0.34      0.36     77361
 

-----------------------------------------------------






Training now:  /doc2vec/Owntrained_withScore.csv
Balanced:  False
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
1517.6253328323364
Best parameters set found on development set:
{'C': 0.1, 'dual': False, 'multi_class': 'ovr', 'tol': 1e-05} 

The model is trained on the full development set.
The scores are computed on the full evaluation set.

Detailed classification report:
              precision    recall  f1-score   support

           0       0.57      1.00      0.73     44096
           1       0.00      0.00      0.00     20237
           2       0.00      0.00      0.00     13028

    accuracy                           0.57     77361
   macro avg       0.19      0.33      0.24     77361
weighted avg       0.32      0.57      0.41     77361
 

-----------------------------------------------------
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



In [8]:
# {'C': 5, 'dual': False, 'multi_class': 'ovr', 'tol': 0.0001}
cross_validation_hyperparameter_grid_search_for_SVM(dataframes, False, 1, 0, description[1][0], param_grid = { 'dual' : [False] ,"tol": [0.0001], 'C': [5], "multi_class": ["ovr"]})

Training now:  /doc2vec/Pretrained_withScore.csv
Balanced:  False
34306.325429201126
Best parameters set found on development set:
{'C': 10, 'dual': False, 'multi_class': 'ovr', 'tol': 0.001} 

The model is trained on the full development set.
The scores are computed on the full evaluation set.

Detailed classification report:
              precision    recall  f1-score   support

           0       0.69      0.92      0.79     44096
           1       0.45      0.17      0.24     20237
           2       0.61      0.50      0.55     13028

    accuracy                           0.65     77361
   macro avg       0.58      0.53      0.53     77361
weighted avg       0.61      0.65      0.60     77361
 

-----------------------------------------------------
